In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"vlb2001","key":"d0e9495b8435d2a615da55cc963c2dd5"}'}

In [ ]:
!rm -r ~/.kaggle
!mkdir ~/.kaggle
!mv ./kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

rm: cannot remove '/root/.kaggle': No such file or directory


In [ ]:
!kaggle competitions download -c uni-na-machine-learning-23-24-ii-sem-contest-3

100% 188M/189M [00:11<00:00, 21.2MB/s]
100% 189M/189M [00:11<00:00, 17.6MB/s]


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/uni-na-machine-learning-23-24-ii-sem-contest-3.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

#Import di librerie

In [ ]:
###########################
# IMPORT
import scipy.io as sio
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import shutil
import copy
import random
from random import shuffle
from torch.utils.data import Dataset
from PIL import Image
import matplotlib.pyplot as plt
######################################


Un primo preprocessing prima dell'elaborazione vera e propria

In [ ]:
# Definisci il percorso della cartella principale "train" e della sottocartella "train"
train_folder = '/content/train'
sub_train_folder = '/content/train/train'
test_folder = '/content/test'
sub_test_folder = '/content/test/test'


#Sposta i file dalla sottocartella alla cartella principale
for filename in os.listdir(sub_train_folder):
    source = os.path.join(sub_train_folder, filename)
    destination = os.path.join(train_folder, filename)
    shutil.move(source, destination)

for filename in os.listdir(sub_test_folder):
    source = os.path.join(sub_test_folder, filename)
    destination = os.path.join(test_folder, filename)
    shutil.move(source, destination)


# Rimuovi la sottocartella vuota "train"
os.rmdir(sub_train_folder)
os.rmdir(sub_test_folder)

print("File del dataset spostati con successo nella cartella 'train'!")

File del dataset spostati con successo nella cartella 'train'!


#Check device


In [ ]:
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

2.3.0+cu121
True
Tesla T4
cuda:0


In [ ]:
#rendere l'esecuzione deterministica
torch.manual_seed(0)
np.random.seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Classe per lettura dati

In [ ]:
class My_DatasetFolder(Dataset):
  def __init__(self, root,  transform, is_valid_file, list_classes):
    self.root = root     #path dei dati
    self.transform = transform    #lista di trasformazioni
    self.is_valid_file = is_valid_file #check per capire quali file leggere
    self.list_classes = list_classes   #lista di classi
    self.samples = self.__get_samples()


  def __len__(self):
    return len(self.samples)

  # def __get_samples(self):
  #   ListFiles=[]
  #   for c in self.list_classes:
  #     list_folder = os.listdir(self.root + '/' + c)
  #     for folder in list_folder:
  #       listofFiles = os.listdir(self.root + '/' + c + '/' + folder)
  #       for file in listofFiles:
  #         if self.is_valid_file(self.root + '/' + c + '/' + folder + '/' + file):
  #           ListFiles.append((self.root + '/' + c + '/' + folder + '/' + file, self.list_classes.index(c)))

  #   return ListFiles

  def __get_samples(self):
    ListFiles=[]
    for c in self.list_classes:
      list_folder = os.listdir(self.root + '/' + c)
      for file in list_folder:
        if self.is_valid_file(self.root + '/' + c + '/' + file):
            ListFiles.append((self.root + '/' + c + '/'+ file, self.list_classes.index(c)))

    return ListFiles

  def loader_fc(self, path):
    img = np.load(path)
    return img

  def __getitem__(self, index: int):
    path, target = self.samples[index]
    sample = self.loader_fc(path)
    #sample = sample.astype(np.float32)    #qui li converte in float32
    if self.transform is not None:
      sample = self.transform(sample)
    return sample, target
      # if target_str is not None:
      #     target = 0 if target_str == 'benign' else 1
      #     return sample, torch.tensor(target)
      # else:
      #     return sample, os.path.basename(path)


'''
un esempio di utilizzo
is_valid_file = lambda path: path.endswith('.npy')

My_DatasetFolder(root = MyPath,  transform = data_transforms, is_valid_file = is_valid_file, list_classes = ['benign', 'malignant'])
'''

"\nun esempio di utilizzo\nis_valid_file = lambda path: path.endswith('.npy')\n\nMy_DatasetFolder(root = MyPath,  transform = data_transforms, is_valid_file = is_valid_file, list_classes = ['benign', 'malignant'])\n"

# Iperparametri

In [ ]:
#iperparametri da scegliere in fase di validation
learning_rate = 0.001
batch_size = 32
set_classes_number = 2
num_epoch = 30

# Funzione di Training

Funzione utilizzata in fase di training, aggiorna i pesi della rete

In [ ]:
# scriviamo riga per riga cosa succede durante un addestramento della rete neurael
# 1 caricare i dati
# 2
# 3 confronto con loss
# 4 aggiornamento del gradiente
# 5

# tutto questo per ogni batch, e lo fai per tanti batch fino ad esaurire dati disponibili, cioè hai finito una epoca


#find best model on validation test
def train_loop_validation(dataloaders, startEpoch, numEpochs, model_conv, criterionCNN, optimizer_conv,
                          best_acc, best_loss, best_epoca, outputPath):

  for epochs in range(startEpoch, numEpochs + 1):
    since = time.time()

    modelLoss_train = 0.0
    modelAcc_train = 0.0

    model_conv.train()

    totalSize = 0


    #for each batch
    for inputs, labels in dataloaders['train']:
      inputs = inputs.type(torch.FloatTensor).cuda()
      labels = labels.cuda()

      # settiamo a zero i gradienti della rete, perchè se no sommi anche quelli precedenti
      optimizer_conv.zero_grad()
      model_conv.zero_grad()

      # output della rete
      y = model_conv(inputs)
      outp, preds = torch.max(y, 1) # il massimo e indice del massimo

      # calcolo la funzione di loss, è un valore solo, la media della loss su un batch
      lossCNN = criterionCNN(y, labels) # utilizzo y vera e propria, la softmax è inclusa nella cross entropy

      modelLoss_train += lossCNN.item() * inputs.size(0) # media * numero di batch: devo aggregare man mano le somme su tutti i batch
      totalSize += inputs.size(0)
      modelAcc_train += torch.sum(preds == labels.data).item()

      lossCNN.backward()  # pred = f(x)   -> loss = L(f(x), l_true)
      optimizer_conv.step()


    modelLoss_epoch_train = modelLoss_train/totalSize # divido somma delle varie loss per la size totale del train, allora ho loss su tutto il train
    modelAcc_epoch_train  = modelAcc_train/totalSize

    #salvataggio dei pesi ad ogni iterazione -> nel caso si blocchi e vogliamo riprendere il train
    torch.save(model_conv.state_dict(), outputPath + 'train_weights.pth')

    model_conv.eval()
    totalSize_val = 0
    modelLoss_val = 0.0
    modelAcc_val = 0.0

    # con il validation sono le stesse cose, tranne per l'aggiornamento dei pesi, perchè dobbiamo solo valutare
    for inputs, labels in dataloaders['val']:
      inputs = inputs.type(torch.FloatTensor).cuda()
      labels = labels.cuda()

      y = model_conv(inputs)
      outp, preds = torch.max(y, 1)
      lossCNN = criterionCNN(y, labels)

      modelLoss_val += lossCNN.item() * inputs.size(0)
      totalSize_val += inputs.size(0)
      modelAcc_val += torch.sum(preds == labels.data).item()


    modelLoss_epoch_val=modelLoss_val/totalSize_val
    modelAcc_epoch_val = modelAcc_val/totalSize_val
    time_elapsed = time.time()-since

    #print(time_elapsed)
    print('[Epoch %d][TRAIN on %d [Loss: %.4f  ACC: %.4f]][VAL on %d [Loss: %.4f  ACC: %.4f]][TIME: %.0f m %.0f s]'
          %(epochs, totalSize, modelLoss_epoch_train, modelAcc_epoch_train, totalSize_val, modelLoss_epoch_val,
            modelAcc_epoch_val, time_elapsed // 60, time_elapsed % 60))

    # questo è un early stopping se mai dovesse risalire la loss del validation
    # così facendo non solo stai salvando i pesi, ma tanto sul validation non stai valutando il modello
    # ti serve solo per salvare l'epoca dopo la quale non migliora, cioè salvarti iperparametro che utilizzerai per il test

    if epochs == 1 or modelLoss_epoch_val < best_loss:
    #if (modelAcc_epoch_val > best_acc) or (modelAcc_epoch_val == best_acc and modelLoss_epoch_val < best_loss) :
      print('     .... Saving best weights ....')
      best_acc = modelAcc_epoch_val
      best_loss = modelLoss_epoch_val
      best_epoca = epochs
      #salvataggio dei migliori pesi sul validation
      torch.save(model_conv.state_dict(), outputPath + 'best_model_weights.pth')

    with open(outputPath + 'lossTrain.txt', "a") as file_object:
      file_object.write(str(modelLoss_epoch_train) +'\n')

    with open(outputPath + 'AccTrain.txt', "a") as file_object:
      file_object.write(str(modelAcc_epoch_train)+'\n')

    with open(outputPath + 'lossVal.txt', "a") as file_object:
      file_object.write(str(modelLoss_epoch_val)+'\n')

    with open(outputPath + 'AccVal.txt', "a") as file_object:
      file_object.write(str(modelAcc_epoch_val)+'\n')

    sio.savemat(outputPath + 'check_point.mat', {'best_acc': best_acc,
                                                 'best_loss': best_loss,
                                                 'best_epoca': best_epoca,
                                                 'last_epoch': epochs})

In [ ]:
# Conteggio di quante sono ogni acquisizione nelle colture

import os

train_path = "/content/train"  # Sostituisci con il percorso corretto

total_images = 0
total_benign = 0
total_malignant = 0
temp_len = 0
len_benign_image = []
len_malignant_image = []
i = 0

for class_folder in sorted(["benign", "malignant"]):  # Itera attraverso le classi
    class_path = os.path.join(train_path, class_folder)
    for acquisition_folder in sorted(os.listdir(class_path)):
        acquisition_path = os.path.join(class_path, acquisition_folder)
        if class_folder == "benign" and i != 0:
            len_benign_image.append(temp_len)
            temp_len = 0
        if class_folder == "malignant" and i != 0:
            len_malignant_image.append(temp_len)
            temp_len = 0
        i += 1
        if os.path.isdir(acquisition_path):
            for image_file in sorted(os.listdir(acquisition_path)):
                if image_file.endswith(".npy"):
                    temp_len += 1
                    if class_folder == "benign":
                        total_benign += 1
                    else:
                        total_malignant += 1
                    total_images += 1

# Append the last temp_len of malignant to len_malignant_image
if i != 0:
    len_malignant_image.append(temp_len)
else:
  total_malignant += 1
total_images += 1
len_benign_image.append(len_malignant_image[0])
len_malignant_image = len_malignant_image[1:]

print(f"Numero totale di immagini nel training set: {total_images}")

print(f"Numero totale di immagini benigne: {total_benign}")
print(f"Numero totale di immagini maligne: {total_malignant}")
print(f"Lista immagini benigne: {len_benign_image}")
print(f"Lista immagini maligne: {len_malignant_image}")

Numero totale di immagini nel training set: 4897
Numero totale di immagini benigne: 1277
Numero totale di immagini maligne: 3619
Lista immagini benigne: [50, 62, 29, 28, 35, 28, 19, 40, 30, 29, 29, 23, 23, 34, 37, 62, 65, 42, 58, 57, 27, 34, 21, 31, 27, 34, 20, 27, 18, 27, 26, 15, 24, 25, 16, 19, 25, 21, 35, 25]
Lista immagini maligne: [110, 86, 115, 87, 80, 74, 73, 87, 83, 75, 93, 83, 65, 75, 83, 104, 83, 109, 71, 88, 97, 98, 71, 109, 94, 100, 92, 86, 90, 79, 77, 85, 115, 90, 87, 101, 97, 94, 105, 128]


# Creazione Cartelle Locali

In [ ]:
import os

nomi_cartelle = ["train_local/benign","train_local/malignant","val_local/benign", "val_local/malignant", "test_local/benign","test_local/malignant"]

for nome in nomi_cartelle:
    # Costruisci il percorso completo della cartella
    percorso_cartella = os.path.join('/content', nome)

    if not os.path.exists(percorso_cartella):
        os.makedirs(percorso_cartella)
        print(f"Cartella '{percorso_cartella}' creata con successo.")
    else:
        print(f"Cartella '{percorso_cartella}' già esistente.")

Cartella '/content/train_local/benign' già esistente.
Cartella '/content/train_local/malignant' già esistente.
Cartella '/content/val_local/benign' già esistente.
Cartella '/content/val_local/malignant' già esistente.
Cartella '/content/test_local/benign' già esistente.
Cartella '/content/test_local/malignant' già esistente.


# Eliminare cartelle

In [ ]:
# import shutil

# # Definisci il nome della cartella che vuoi eliminare
# nome_cartella = "/content/train_local/malignant"

# # Elimina la cartella se esiste
# if os.path.exists(nome_cartella):
#     shutil.rmtree(nome_cartella)
#     print(f"Cartella '{nome_cartella}' eliminata con successo.")
# else:
#     print(f"Cartella '{nome_cartella}' non trovata.")

In [ ]:

# Percentuale di benign e malign sull'intero dataset
percent_benign_tot = total_benign/total_images
percent_malign_tot = 1 - percent_benign_tot

print(percent_benign_tot)
print(percent_malign_tot)

# Percentuale di training, val e test set su intero training set locale
len_train_local = total_images*70/100
len_val_local = total_images*15/100
len_test_local = len_val_local

print(len_train_local)
print(len_val_local)
print(len_test_local)

mal_treshold_train_local = len_train_local * percent_malign_tot
mal_treshold_val_local = len_val_local * percent_malign_tot
mal_treshold_test_local = len_val_local

ben_treshold_train_local = len_train_local * percent_benign_tot
ben_treshold_val_local = len_val_local * percent_benign_tot
mal_treshold_test_local = len_val_local

print(mal_treshold_train_local)
print(ben_treshold_train_local)

# Calcolo del numero di immagini maligne e benigne per training, val e test set
num_malign_train = 0
num_benign_train = 0
num_malign_val = 0
num_benign_val = 0
num_malign_test = 0
num_benign_test = 0

import os
import shutil

# Define the paths for train, val, and test folders
train_folder = "/content/train_local"
val_folder = "/content/val_local"
test_folder = "/content/test_local"

# Create the train, val, and test folders if they don't exist
os.makedirs(train_folder, exist_ok=True)
os.makedirs(val_folder, exist_ok=True)
os.makedirs(test_folder, exist_ok=True)

# Move the images to the train, val, and test folders based on the calculated thresholds
for class_folder in sorted(["benign", "malignant"]):
    class_path = os.path.join(train_path, class_folder)
    for acquisition_folder in sorted(os.listdir(class_path)):
        acquisition_path = os.path.join(class_path, acquisition_folder)
        if os.path.isdir(acquisition_path):
            for image_file in sorted(os.listdir(acquisition_path)):
                if image_file.endswith(".npy"):
                    if class_folder == "malignant":
                        if num_malign_train < mal_treshold_train_local:
                            destination_folder = os.path.join(train_folder, class_folder)
                            num_malign_train += 1
                        elif num_malign_val < mal_treshold_val_local:
                            destination_folder = os.path.join(val_folder, class_folder)
                            num_malign_val += 1
                        else:
                            destination_folder = os.path.join(test_folder, class_folder)
                            num_malign_test += 1
                    else:  # class_folder == "benign"
                        if num_benign_train < ben_treshold_train_local:
                            destination_folder = os.path.join(train_folder, class_folder)
                            num_benign_train += 1
                        elif num_benign_val < ben_treshold_val_local:
                            destination_folder = os.path.join(val_folder, class_folder)
                            num_benign_val += 1
                        else:
                            destination_folder = os.path.join(test_folder, class_folder)
                            num_benign_test += 1

                    # Create the destination folder if it doesn't exist
                    os.makedirs(destination_folder, exist_ok=True)

                    source_path = os.path.join(acquisition_path, image_file)
                    destination_path = os.path.join(destination_folder, image_file)
                    shutil.copyfile(source_path, destination_path)


# Print the number of images in each set
print(f"Number of malignant images in train set: {num_malign_train}")
print(f"Number of benign images in train set: {num_benign_train}")
print(f"Number of malignant images in val set: {num_malign_val}")
print(f"Number of benign images in val set: {num_benign_val}")
print(f"Number of malignant images in test set: {num_malign_test}")
print(f"Number of benign images in test set: {num_benign_test}")


0.26077190116397797
0.739228098836022
3427.9
734.55
734.55
2534.0
893.9000000000001
Number of malignant images in train set: 2534
Number of benign images in train set: 894
Number of malignant images in val set: 543
Number of benign images in val set: 192
Number of malignant images in test set: 542
Number of benign images in test set: 191


# Caricamento dei Dati

In [ ]:
# varie trasformazioni che vengono applicate alle immagini prima che verranno caricate in memoria
# trasformazioni su train e val sono diverse, hanno in comune quella in tensor, ma serve solo per formato giusto ingresso rete
# classe normalize prende in ingresso due vettori, uno delle medie e uno delle deviazioni standard, ogni vettore è da 3 perchè abbiamo tre canali
# è come se avessi fatto ogni canale a media nulla e deviaz std unitaria, i numeri sono stati ottenuti calcolando le medie e dev std sul training
# quindi quando lo si fa nel pratico devi calcolarteli prima a parte

data_transforms = {
    'train_local': transforms.Compose([ # con augumentation irrobustico il sistema, applico delle variabilità per cercare di introdurre generalità nel modello
        transforms.ToTensor(), # converto immagine in tensore
        transforms.Resize(224),
        #transforms.ToPILImage(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # qusti sono quelli ottenuti per dataset iniziale che noi non abbiamo, medie e dev di imagenet (il dataset originale)
    ]),
    'val_local': transforms.Compose([ # facciamo una cosa controllata sul val e non randomica perchè bisogna fare una prima valutazione sul validation
        transforms.ToTensor(),
        transforms.Resize(224),
        #transforms.ToPILImage(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test_local': transforms.Compose([ # facciamo una cosa controllata sul val e non randomica perchè bisogna fare una prima valutazione sul validation
        transforms.ToTensor(),
        transforms.Resize(224),
        #transforms.ToPILImage(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# image folder si ricorda dove stanno le immagini senza caricarle in memoria
# gestiamo in maniera efficiente i nostri dati, in particolare i percorrisi delle immagini
# in attributo samples ci sono tutti i path e le classi associate
# in attributo classes ci sono il vettore delle classi, in ordine di indice come sono mostrate
# per accedere all'immagine vera e propria la si ha con funzione getitem, così la carichi proprio
# però prima che prendiamo un immagine in mem, image folder la converte con le trasformazioini di sopra
# ogni immagine in pytorch deve essere un tensore per poter essere elaborata
# 1d = numero canali, 2d e 3d grandezza immagine

content_dir = '/content'

is_valid_file = lambda path: path.endswith('.npy')
my_dataset = {x: My_DatasetFolder(os.path.join(content_dir, x),
                                          data_transforms[x],is_valid_file=is_valid_file,list_classes=['benign','malignant'])
                  for x in ['train_local','val_local','test_local']}



# # Il dataloader è la classe responsabile per caricare i dati in memoria

dataloaders = {x: torch.utils.data.DataLoader(my_dataset[x], batch_size=batch_size,
                                             shuffle=True, num_workers=4)
              for x in ['train_local','val_local','test_local']}
dataset_sizes = {x: len(my_dataset[x]) for x in ['train_local','val_local','test_local']}
class_names = my_dataset['train_local'].list_classes
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# print(class_names)
# print(image_datasets['val'].classes)

# # Quando sono stati caricati i dati in mem lo si è fatto in due cartelle separate, una per training e una per validation
# # perciò sono stati fatti due imagefolder


In [ ]:
my_dataset['test_local'].samples



[('/content/test_local/benign/mcf10aCdExposed-40_croppedCell_17.npy', 0),
 ('/content/test_local/benign/mcf10aCdExposed-40_croppedCell_22.npy', 0),
 ('/content/test_local/benign/mcf10aCdExposed-37_croppedCell_6.npy', 0),
 ('/content/test_local/benign/mcf10aCdExposed-37_croppedCell_15.npy', 0),
 ('/content/test_local/benign/mcf10aCdExposed-34_croppedCell_13.npy', 0),
 ('/content/test_local/benign/mcf10aCdExposed-33_croppedCell_1.npy', 0),
 ('/content/test_local/benign/mcf10aCdExposed-37_croppedCell_11.npy', 0),
 ('/content/test_local/benign/mcf10aCdExposed-37_croppedCell_14.npy', 0),
 ('/content/test_local/benign/mcf10aCdExposed-33_croppedCell_17.npy', 0),
 ('/content/test_local/benign/mcf10aCdExposed-36_croppedCell_14.npy', 0),
 ('/content/test_local/benign/mcf10aCdExposed-38_croppedCell_19.npy', 0),
 ('/content/test_local/benign/mcf10aCdExposed-34_croppedCell_12.npy', 0),
 ('/content/test_local/benign/mcf10aCdExposed-37_croppedCell_12.npy', 0),
 ('/content/test_local/benign/mcf10aCdEx

# Training Function

In [ ]:
def train_loop_validation(dataloaders, startEpoch, numEpochs, model_conv, criterionCNN, optimizer_conv,
                          best_acc, best_loss, best_epoca, outputPath):

  for epochs in range(startEpoch, numEpochs + 1):
    since = time.time()

    modelLoss_train = 0.0
    modelAcc_train = 0.0

    model_conv.train()

    totalSize = 0


    #for each batch
    for inputs, labels in dataloaders['train_local']:
      inputs = inputs.type(torch.FloatTensor).cuda()
      labels = labels.cuda()

      # settiamo a zero i gradienti della rete, perchè se no sommi anche quelli precedenti
      optimizer_conv.zero_grad()
      model_conv.zero_grad()

      # output della rete
      y = model_conv(inputs)
      outp, preds = torch.max(y, 1) # il massimo e indice del massimo

      # calcolo la funzione di loss, è un valore solo, la media della loss su un batch
      lossCNN = criterionCNN(y, labels) # utilizzo y vera e propria, la softmax è inclusa nella cross entropy

      modelLoss_train += lossCNN.item() * inputs.size(0) # media * numero di batch: devo aggregare man mano le somme su tutti i batch
      totalSize += inputs.size(0)
      modelAcc_train += torch.sum(preds == labels.data).item()

      lossCNN.backward()  # pred = f(x)   -> loss = L(f(x), l_true)
      optimizer_conv.step()


    modelLoss_epoch_train = modelLoss_train/totalSize # divido somma delle varie loss per la size totale del train, allora ho loss su tutto il train
    modelAcc_epoch_train  = modelAcc_train/totalSize

    #salvataggio dei pesi ad ogni iterazione -> nel caso si blocchi e vogliamo riprendere il train
    torch.save(model_conv.state_dict(), outputPath + 'train_weights.pth')

    model_conv.eval()
    totalSize_val = 0
    modelLoss_val = 0.0
    modelAcc_val = 0.0

    # con il validation sono le stesse cose, tranne per l'aggiornamento dei pesi, perchè dobbiamo solo valutare
    for inputs, labels in dataloaders['val_local']:
      inputs = inputs.type(torch.FloatTensor).cuda()
      labels = labels.cuda()

      y = model_conv(inputs)
      outp, preds = torch.max(y, 1)
      lossCNN = criterionCNN(y, labels)

      modelLoss_val += lossCNN.item() * inputs.size(0)
      totalSize_val += inputs.size(0)
      modelAcc_val += torch.sum(preds == labels.data).item()


    modelLoss_epoch_val=modelLoss_val/totalSize_val
    modelAcc_epoch_val = modelAcc_val/totalSize_val
    time_elapsed = time.time()-since

    #print(time_elapsed)
    print('[Epoch %d][TRAIN on %d [Loss: %.4f  ACC: %.4f]][VAL on %d [Loss: %.4f  ACC: %.4f]][TIME: %.0f m %.0f s]'
          %(epochs, totalSize, modelLoss_epoch_train, modelAcc_epoch_train, totalSize_val, modelLoss_epoch_val,
            modelAcc_epoch_val, time_elapsed // 60, time_elapsed % 60))

    # questo è un early stopping se mai dovesse risalire la loss del validation
    # così facendo non solo stai salvando i pesi, ma tanto sul validation non stai valutando il modello
    # ti serve solo per salvare l'epoca dopo la quale non migliora, cioè salvarti iperparametro che utilizzerai per il test

    if epochs == 1 or modelLoss_epoch_val < best_loss:
    #if (modelAcc_epoch_val > best_acc) or (modelAcc_epoch_val == best_acc and modelLoss_epoch_val < best_loss) :
      print('     .... Saving best weights ....')
      best_acc = modelAcc_epoch_val
      best_loss = modelLoss_epoch_val
      best_epoca = epochs
      #salvataggio dei migliori pesi sul validation
      torch.save(model_conv.state_dict(), outputPath + 'best_model_weights.pth')

    with open(outputPath + 'lossTrain.txt', "a") as file_object:
      file_object.write(str(modelLoss_epoch_train) +'\n')

    with open(outputPath + 'AccTrain.txt', "a") as file_object:
      file_object.write(str(modelAcc_epoch_train)+'\n')

    with open(outputPath + 'lossVal.txt', "a") as file_object:
      file_object.write(str(modelLoss_epoch_val)+'\n')

    with open(outputPath + 'AccVal.txt', "a") as file_object:
      file_object.write(str(modelAcc_epoch_val)+'\n')

    sio.savemat(outputPath + 'check_point.mat', {'best_acc': best_acc,
                                                 'best_loss': best_loss,
                                                 'best_epoca': best_epoca,
                                                 'last_epoch': epochs})

# Scelta del Modello

In [ ]:
'''
Inserire il modello che si vuole considerare
'''

networkName = 'vgg11'
WeightPath = '/content/gdrive/My Drive/Colab Notebooks/' + networkName + '/'

try:
  os.makedirs(WeightPath)
except:
  pass

'''
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model_ft.fc = nn.Linear(num_ftrs, 2)

for param in model_ft.parameters():
  param.requires_grad = True

'''

model = torchvision.models.vgg11(pretrained=True)
# remove last fully-connected layer


# change last fully connected layer
model.classifier[6] = nn.Linear(4096, set_classes_number)


# set requires_grad=false
for param in model.parameters():
  param.requires_grad = False # tutta la rete si sblocca

# for layer in model.features[18:]:
#   for param in layer.parameters():
#       param.requires_grad = True

# train fully connected layers
for param in model.classifier.parameters():
  param.requires_grad = True # sblocchiamo i layer della parte di classificazione, si aggiornano durante l'addestramento

print(model.parameters)


# Ora vogliamo addestrare, per prima cosa bisogna scegliere la funzione di loss
# per le classi va per la maggiore la cross entropy/ binary cross entropy
# ci serve anche un ottimizzatore, cioè quello che effettivamente fa algoritmo iterativo che fa gli step

model_conv = model.cuda()
criterion = nn.CrossEntropyLoss()
optimizer_conv = optim.Adam(model_conv.parameters(), lr=learning_rate)

# -------------------------------------------------------------------------------------  train model
startEpoch =1
best_acc = 0
best_loss=0
best_epoca = 0

# funzione definita precedentemente per fare il training
train_loop_validation(dataloaders, startEpoch, num_epoch, model_conv, criterion, optimizer_conv, best_acc, best_loss, best_epoca, WeightPath)

<bound method Module.parameters of VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace=True)
    (15): MaxPool2d(kernel_size=2, stride=2, paddin

# Feature Extraction

In [ ]:
outfeatures_train =[]

data_transforms = transforms.Compose([
                                      transforms.Resize(224),
                                      transforms.ToTensor(),
                                         #questo è solo un esempio!
                                      ])

'''
Inserire il modello che si vuole considerare
'''
model = torchvision.models.vgg11(pretrained=True)
# remove last fully-connected layer
new_classifier = nn.Sequential(*list(model.classifier.children())[:-1])
model.classifier = new_classifier
model = model.cuda()
model.eval()

class_list_train = []
class_list_test = []
classes = ['benign', 'malignant']
data_dir = '/content'   #path dei dati

for c in classes:
  folder = os.listdir(data_dir + '/train/' + c)
  for f in folder:
    listImage = os.listdir(data_dir + '/train/' + c + '/' + f)
    for input in listImage:
      img = np.load(data_dir + '/train/' + c + '/' + f + '/' + input)
      img_pil = Image.fromarray((img * 255).astype(np.uint8))
      img = data_transforms(img_pil).type(torch.FloatTensor).to(device)
      img = img.unsqueeze_(0)
      outputs = model(img)

      outfeatures_train.append((np.squeeze(outputs.cpu().detach()).numpy()))
      class_list_train.append(classes.index(c))

# Dividiamo il training test in training, validation e test locale



/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG11_Weights.IMAGENET1K_V1`. You can also use `weights=VGG11_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg11-8a719046.pth" to /root/.cache/torch/hub/checkpoints/vgg11-8a719046.pth
100%|██████████| 507M/507M [00:06<00:00, 80.5MB/s]
